<table style="float:left; border:none">
   <tr style="border:none; background-color: #ffffff">
       <td style="border:none">
           <a href="http://www.islas.org.mx//">     
           <img 
               src="assets/geci_20_anos.png" 
               style="width:200px"
           >
           </a>    
       </td>
       <td style="border:none">
           <h1>Erradicación de gatos en Socorro</h1>
       </td>
   </tr>
</table>

<div style="float:right;"><h2> Efecto del esfuerzo</h2></div>

Evaluaremos el efecto que tiene usar diferentes tamaños de esfuerzo (_unidad de esfuerzo_) en las simulaciones. Lo que se
quiere hacer es una gráfica de las capturas en función del esfuerzo. Dos de las menras en las que podemos hacer la gráfica son:
1. Calcular capturas para distintos esfuerzos
1. Hacer "series de tiempo" de las salidas

## Paqueterías 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gatos as dmc
import bokeh.plotting as bp
from math import ceil

## Datos
Los datos que usaremos son las distribuciones posteriores de los parámetros del modelo de Ramsey obtenidas del ajuste a partir
de los datos de cacería de gatos en Isla Socorro.

Los parámetros son: 
1. la población inicial $No$
1. $\alpha$ y $\beta$ que definen la probabilidad de captura en función del esfuerzo. 

In [2]:
directorio_resultados = '../tmp/'
archivo = 'distribucion_posterior_repeticion_2_iteraciones_1000000.csv'
datos = pd.read_csv(directorio_resultados + archivo)

## Capturas a un esfuerzo dado


### El esfuerzo en una sola salida (un esfuerzo grande)

Tomaremos una muestra de 10000 de las distribuciones posteriores. Con cada una de esas muestras simulamos eventos de capturas. 
Usaremos los resultados de las capturas (`capturas_esfuerzo_grande`) para hacer un histograma. 

In [3]:
muestra = 100_000
esfuerzo = 20

In [4]:
No = datos.No.sample(n=muestra, replace=True).values.astype(np.int32) - 606
beta = datos.b.sample(n=muestra, replace=True).values
alpha = datos.a.sample(n=muestra, replace=True).values
exp = np.exp(alpha + beta * esfuerzo)
p = exp/(1+exp)
capturas_esfuerzo_grande = np.random.binomial(No,p)

### El esfuerzo en varias salidas (muchos esfuerzos chicos)
Simularemos eventos de cacería, en cada evento haremos un esfuerzo de 1. Por lo tanto, deberemos tener `nIntentos` para alcanzar
el esfuerzo total. Repetiremos la simulación para cada uno de las ternas ($\alpha$, $\beta$ y $N_o$) arriba. Usaremos los 
resultados de las capturas (`capturas_esfuerzo_chico`) para hacer un histograma.

In [5]:
unidad_esfuerzo = 0.1

In [6]:
nIntentos = int(esfuerzo/unidad_esfuerzo) 
esfuerzo_chico = np.random.uniform(unidad_esfuerzo,unidad_esfuerzo,nIntentos)
capturas_esfuerzo_chico = []
for iRepeticion in range(muestra):
    vector_probabilidad = dmc.calculateProbabilityFromAlphaBetaEffort(alpha[iRepeticion], beta[iRepeticion], esfuerzo_chico)        
    probabilidadEmpirica = dmc.CalculaProbabilidadEmpirica(vector_probabilidad)
    tamanoPoblacion = No[iRepeticion] 
    gatosRemovidos = dmc.getRemovedCats(tamanoPoblacion,probabilidadEmpirica,nIntentos)
    capturas_esfuerzo_chico.append(sum(gatosRemovidos))

## Graficado
Estimamos el número de particiones que debería de tener cada histograma, para que cada separación corresponda a un posible valor 
de $N_o$.

In [7]:
bins_grande = max(capturas_esfuerzo_grande)-min(capturas_esfuerzo_grande)

In [8]:
bins_chico = max(capturas_esfuerzo_chico)-min(capturas_esfuerzo_chico)

Calculamos los parámetros de los histogramas:

In [9]:
histograma_grande, lados_grande, basura = plt.hist(capturas_esfuerzo_grande, bins= bins_grande, normed=1)
histograma_chico, lados_chico, basura   = plt.hist(capturas_esfuerzo_chico,  bins= bins_chico, normed=1)

Definimos las opciones del la figura:

In [10]:
titulo_hitograma = "Diferencias debidas al esfuerzo (Esfuerzo total = {})"
titulo_datos_esfuerzo = "Muchos esfuerzos chicos (Unidad de esfuerzo = {})"
limite_x = max(bins_grande,bins_chico)
limite_y = max(max(histograma_grande),max(histograma_chico))*100
limite_y = ceil(limite_y)/100
opciones_grafica = dict(title=titulo_hitograma.format(esfuerzo), x_axis_label='Gatos retirados', y_axis_label='Probabilidad', \
            y_range=(0,limite_y), x_range = (0,limite_x))

Graficamos utilizando los parámetros de los histogramas y las opciones de la figura: 

In [11]:
p = bp.figure(**opciones_grafica)
p.quad(top=histograma_grande, bottom=0, left=lados_grande[:-1], right=lados_grande[1:],legend="Un esfuerzo grande")
p.quad(top=histograma_chico, bottom=0, left=lados_chico[:-1], right=lados_chico[1:],fill_color= "#D95B43", alpha = 0.5,\
       legend=titulo_datos_esfuerzo.format(unidad_esfuerzo))
p.grid.visible = False

Guardamos y mostramos la figura:

In [12]:
bp.output_file('../resultados/comparacion_esfuerzo.html', title="gatos retirados diferentes esfuerzos")
bp.show(p)